<a href="https://colab.research.google.com/github/SaintKyou/ML/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22tuner_params_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
data = pd.read_csv('output.csv')
data[:5]

,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,tet,phi,kr,X,Y,Ne
0,37.48510,21.47650,72.03370,74.87260,0.00000,7.15842,2.86372,9.73767,3.95813,6.82865,20.3750,7.36746,0.0000,10.39340,7.18657,0.00000,0.0,0.00000,2.69125,2.48769,4.6243,18.67210,1.56277,6.99705,2.01758,4.62862,0.00000,9.33522,1.61079,4.89950,0.00000,5.03119,19.24120,18.36110,6.50392,9.45598,8.83209,222.3430,9.0,-33.7909,-10.0577,33905.80
1,20.08870,7.55648,28.97700,44.00660,2.01309,0.00000,8.88389,4.70197,4.70138,9.87103,11.0701,4.42813,11.1692,5.97066,5.42396,0.00000,0.0,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,2.03481,2.33048,0.00000,1.63118,29.05450,0.00000,0.00000,25.82190,13.31970,5.86606,0.00000,40.72400,155.3300,7.0,-33.7909,-10.0577,22573.60
2,32.82920,144.71500,246.86400,83.69460,10.50480,28.81570,28.77490,36.16710,7.96149,24.13200,22.7822,7.48365,0.0000,7.35025,2.71817,5.37989,0.0,2.72421,2.46114,4.67105,20.8686,6.82867,3.77993,4.43100,2.79168,0.00000,2.60397,4.06729,3.08867,2.98717,2.83806,2.08980,56.50520,34.54030,27.97570,32.25710,45.21980,301.3900,9.0,-33.7909,-10.0577,65195.60
3,6.48722,0.00000,2.09672,9.09409,2.44893,8.25071,10.60860,0.00000,0.00000,0.00000,0.0000,0.00000,0.0000,0.00000,0.00000,0.00000,0.0,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,9.06438,8.90096,0.00000,48.65310,74.7579,3.0,-33.7909,-10.0577,5262.08
4,12.42410,7.65120,11.86360,38.41840,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0000,0.00000,0.0000,1.71730,0.00000,1.54347,0.0,0.00000,0.00000,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,9.78031,8.81661,1.78242,3.93061,43.13070,187.1960,3.0,-33.7909,-10.0577,11204.50


In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 71kB 3.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=b92c0d24be78dd4ab8aba1100899cd4eb8cf5b4ac123b6d34e54a6b56142b111
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15357 sha256=2a80c00bc6a8a2dff364e10386af031dc505bce7c6b5531587196373a9cc7fd7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from tensorflow.keras import regularizers
from google.colab import files
from kerastuner.tuners import BayesianOptimization
import numpy as np

In [ ]:
df = data[data.kr>6]
df = df[df.X>=-37.55] # отбираем события внутри установки
df = df[df.X<=42.13]
df = df[df.Y>=-70.50]
df = df[df.Y<=64.66]
df = df.drop(columns=["kr"])
df = df.drop(columns=["Ne"])
df = df.drop(columns=["X"])
from sklearn.model_selection import train_test_split

y = df["Y"]
X = df.drop(columns=["Y"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [ ]:
# Среднее значение
mean = X_train.mean(axis=0)
# Стандартное отклонение
std = X_train.std(axis=0)
X_train -= mean
X_train /= std
X_test -= mean
X_test /= std

In [ ]:
from tensorflow.keras import initializers

def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):  # кол-во слоев (мин, макс)
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=38,
                                            max_value=512,
                                            step=32),
                               activation='relu', activity_regularizer=regularizers.l2(1e-4), kernel_initializer='he_uniform')) 
    model.add(Dense(1,kernel_initializer='glorot_uniform')) # число нейронов на выходном слое
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam']),
        loss='mse',
        metrics=['mae'])
    return model

In [ ]:
tuner = BayesianOptimization(
    build_model,                 # функция создания модели
    objective='mae',             # метрика, которую нужно оптимизировать - 
                                 
    max_trials=80,               # максимальное количество запусков обучения 
    directory='abcd'   # каталог, куда сохраняются обученные сети  
    )

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 38, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 38, 'max_value': 512, 'step': 32, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam'], 'ordered': False}


In [ ]:
tuner.search(X_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Trial 80 Complete [00h 00m 55s]
mae: 4.016036033630371

Best mae So Far: 3.5370845794677734
Total elapsed time: 01h 20m 00s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary() # результаты

Results summary
Results in abcd/untitled_project
Showing 10 best trials
Objective(name='mae', direction='min')
Trial summary
Hyperparameters:
num_layers: 20
units_0: 486
units_1: 486
optimizer: adam
units_2: 486
units_3: 486
units_4: 486
units_5: 486
units_6: 38
units_7: 486
units_8: 38
units_9: 38
units_10: 38
units_11: 486
units_12: 38
units_13: 486
units_14: 38
units_15: 486
units_16: 38
units_17: 38
units_18: 486
units_19: 486
Score: 3.5370845794677734
Trial summary
Hyperparameters:
num_layers: 20
units_0: 486
units_1: 486
optimizer: adam
units_2: 486
units_3: 486
units_4: 38
units_5: 38
units_6: 38
units_7: 38
units_8: 486
units_9: 38
units_10: 38
units_11: 38
units_12: 486
units_13: 38
units_14: 38
units_15: 38
units_16: 486
units_17: 38
units_18: 486
units_19: 486
Score: 3.606062412261963
Trial summary
Hyperparameters:
num_layers: 20
units_0: 486
units_1: 486
optimizer: adam
units_2: 486
units_3: 486
units_4: 486
units_5: 486
units_6: 486
units_7: 38
units_8: 38
units_9: 38
unit

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0] # лучшая сеть

In [ ]:
mdl = tuner.hypermodel.build(best_hps) # создаем сеть с лучшими гиперпараметрами

In [ ]:
from keras.callbacks import EarlyStopping
mdl.fit(X_train, y_train, epochs=50, batch_size=128, verbose=2)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

99/99 - 4s - loss: 1064.4952 - mae: 25.3405
Epoch 2/50
99/99 - 4s - loss: 214.0102 - mae: 10.6341
Epoch 3/50
99/99 - 4s - loss: 216.5816 - mae: 10.1863
Epoch 4/50
99/99 - 4s - loss: 122.5763 - mae: 7.5310
Epoch 5/50
99/99 - 4s - loss: 92.3801 - mae: 6.4846
Epoch 6/50
99/99 - 4s - loss: 68.4949 - mae: 5.6034
Epoch 7/50
99/99 - 4s - loss: 53.9331 - mae: 5.0385
Epoch 8/50
99/99 - 4s - loss: 62.0482 - mae: 5.4142
Epoch 9/50
99/99 - 4s - loss: 46.6390 - mae: 4.6750
Epoch 10/50
99/99 - 4s - loss: 4

In [ ]:
mdl.save("y_po.h5") # сохраняем сеть